In [1]:
# lovasz softmax
from loss.lovasz_softmax import lovasz_softmax

In [ ]:
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from PIL import Image

class CustomDataset(Dataset):

  def __init__(self, baseDataset):
    self.ds = baseDataset
    self.transform = transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p = 0.5),
    transforms.RandomApply([transforms.RandomRotation(degrees=10)], p = 0.5),
    transforms.ToTensor()
    ])


  def __len__(self):
    return len(self.ds)

  def __getitem__(self, idx):
    image, label = self.ds[idx]
    image = Image.fromarray((image * 255).astype('uint8'))  # Assuming array is scaled [0, 1]
    image = self.transform(image)
    return image, label



In [ ]:
import torch
import torch.nn.functional as F

class WeightedCrossEntropyLoss(torch.nn.Module):
    def __init__(self, class_weights):
        super().__init__()
        self.class_weights = class_weights

    def forward(self, inputs, targets):
        return F.cross_entropy(inputs, targets, weight = self.class_weights)

class CombinedLoss(torch.nn.Module):
    def __init__(self, class_weights):
        super().__init__()
        self.wce_loss = WeightedCrossEntropyLoss(class_weights)
        self.lovasz_loss = lovasz_softmax

    def forward(self, inputs, targets):
        loss_wce = self.wce_loss(inputs, targets)
        loss_lovasz = self.lovasz_loss(inputs, targets)
        return loss_wce + loss_lovasz

In [ ]:
from torch.optim import SGD
from torch.optim.lr_scheduler import StepLR

def train_model(model, train_loader, class_weights, num_epochs, is_loss_graph = False):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    optimizer = SGD(model.parameters(), lr = 0.01, momentum= 0.9, weight_decay = 0.0001)
    scheduler = StepLR(optimizer, 1, gamma = 0.01)

    loss_fn = CombinedLoss(class_weights)
    loss_values = []

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        scheduler.step()

        epoch_loss = total_loss / len(train_loader)

        if is_loss_graph:
            loss_values.append(epoch_loss)

        print(f'Epoch {epoch + 1}, Loss: {epoch_loss}')

    if is_loss_graph:
      plt.plot(range(1, num_epochs + 1), loss_values)
      plt.xlabel('Epoch')
      plt.ylabel('Loss')
      plt.title('Training Loss')
      plt.show()

In [ ]:
def get_class_weights(dataloader, num_classes = 20):
    class_counts = {}

    for _, labels in dataloader:
        # assuming label is int
        for label in labels:
            class_counts[label] = class_counts.get(label, 0) + 1


    class_frequencies = torch.zeros(num_classes)
    for key, value in class_counts.items():
        class_frequencies[key] = value

    weights = 1.0 / torch.sqrt(class_frequencies)
    weights[class_frequencies == 0] = 0

    return weights


In [ ]:
from torch.utils.data import DataLoader

trainset = None # TODO
testset = None #TODO

batch_size = 24

train_loader = DataLoader(CustomDataset(trainset), batch_size=batch_size,
                                          shuffle=True, num_workers=2)
testloader = DataLoader(CustomDataset(testset), batch_size=batch_size,
                                         shuffle=False, num_workers=2)

model = None # TODO: create model instance


num_epochs = 10 # TODO: play with this hyperparameter
class_weights = get_class_weights(train_loader)
train_model(model, train_loader, class_weights, num_epochs)

TypeError: object of type 'NoneType' has no len()